In [1]:
# %load src/cropper.py
from tqdm import trange
import numpy as np
from src.seam_table_service import SeamTableService
from src.energy_carving import Energy
import visvis as vv
i_energy = Energy()


class Cropper():
    @staticmethod
    def crop_c(img, scale_c):
        r, c, _ = img.shape
        new_c = int(scale_c * c)
        for i in trange(c - new_c):
            energy_map = i_energy.calc_energy(img)
            status, img = SeamTableService.carve_column(img, energy_map)

        return img

    @staticmethod
    def remove_object(img, object_point):
        r, c, _ = img.shape

        if len(object_point) > 0:
            object_removal_mask = SeamTableService.build_mask_with_object_removal(r, c, object_point)
            vv.imshow(object_removal_mask)
            isFinish = False
            while not isFinish:
                energy_map = i_energy.calc_energy(img)
                energy_map_fixed = SeamTableService.energy_map_with_concern_to_object(energy_map, object_removal_mask)
                isFinish, img, object_removal_mask = SeamTableService.carve_column(img=img, energy_map=energy_map_fixed,
                                                                    mode='object_removal', object_removal_mask=object_removal_mask)

        return img

    @staticmethod
    def crop_r(img, scale_r):
        img = np.rot90(img, 1, (0, 1))
        img = SeamTableService.crop_c(img, scale_r)
        img = np.rot90(img, 3, (0, 1))
        return img


In [2]:
# %load src/energy_carving.py
import numpy as np
from scipy.ndimage.filters import convolve

class Energy():
    def __init__(self):
        filter_du = np.array([
            [1.0, 2.0, 1.0],
            [0.0, 0.0, 0.0],
            [-1.0, -2.0, -1.0],
        ])
        # This converts it from a 2D filter to a 3D filter, replicating the same
        # filter for each channel: R, G, B
        self.filter_du = np.stack([filter_du] * 3, axis=2)

        filter_dv = np.array([
            [1.0, 0.0, -1.0],
            [2.0, 0.0, -2.0],
            [1.0, 0.0, -1.0],
        ])
        # This converts it from a 2D filter to a 3D filter, replicating the same
        # filter for each channel: R, G, B
        self.filter_dv = np.stack([filter_dv] * 3, axis=2)
    def calc_energy(self,img):
        img = img.astype('float32')
        convolved = np.absolute(convolve(img, self.filter_du)) + np.absolute(convolve(img, self.filter_dv))
        # We sum the energies in the red, green, and blue channels
        energy_map = convolved.sum(axis=2)

        return energy_map

In [4]:
# %load src/seam_table_service.py
import numpy as np

np.bitwise_not is np.invert
class SeamTableService():

    @staticmethod
    def build_minimum_seam_table(r, c, energy_map):
        M = energy_map.copy()
        backtrack = np.zeros_like(M, dtype=np.int)

        for i in range(1, r):
            for j in range(0, c):
                # Handle the left edge of the image, to ensure we don't index -1
                if j == 0:
                    idx = np.argmin(M[i - 1, j:j + 2])
                    backtrack[i, j] = idx + j
                    min_energy = M[i - 1, idx + j]
                else:
                    idx = np.argmin(M[i - 1, j - 1:j + 2])
                    backtrack[i, j] = idx + j - 1
                    min_energy = M[i - 1, idx + j - 1]

                M[i, j] += min_energy

        return M, backtrack

    @staticmethod
    def build_mask_with_object_removal(r, c, object_point):
        mask = np.zeros((r, c), dtype=np.bool)
        for point in object_point:
            # x = point[0] , y = point[1]
            mask[point[0], point[1]] = True
        return mask

    @staticmethod
    def energy_map_with_concern_to_object(energy_map, mask):
        energy_map[mask] = -10000
        return energy_map

    @staticmethod
    def carve_column(img, energy_map, mode=None, object_removal_mask=None):
        r, c, _ = img.shape
        M, backtrack = SeamTableService.build_minimum_seam_table(r, c, energy_map)

        # Find the position of the smallest element in the
        # last row of M
        j = np.argmin(M[-1])

        if mode == 'object_removal':
            print("the minumum path is :{}".format(M[-1, j]))
            if M[-1, j] > 0:
                return True, img ,object_removal_mask

        # Create a (r, c) matrix filled with the value True
        # We'll be removing all pixels from the image which
        # have False later
        mask = np.ones((r, c), dtype=np.bool)

        for i in reversed(range(r)):
            # Mark the pixels for deletion
            mask[i, j] = False
            j = backtrack[i, j]
        # Since the image has 3 channels, we convert our
        # mask to 3D
        if mode == 'object_removal':
            object_removal_mask = object_removal_mask[mask].reshape((r, c - 1))

        mask = np.stack([mask] * 3, axis=2)

        # Delete all the pixels marked False in the mask,
        # and resize it to the new image dimensions
        img = img[mask].reshape((r, c - 1, 3))


        return False, img, object_removal_mask


def remove_redundent_point(minmum_path_indexes, object_point):
    if len(minmum_path_indexes) > 0:
        minmum_path_indexes_dict = dict(minmum_path_indexes)
        print("len of removal objects is :{}".format(len(object_point)))
        for point in object_point:
            point_minumum_index = minmum_path_indexes_dict.get(point[0])
            if (point_minumum_index is not None) and (point_minumum_index == point[1]):
                object_point.remove(point)
        print("len of removal objects is: {}".format(len(object_point)))
    return object_point


In [5]:
from imageio import imread, imwrite
import os

base_dir =os.path.abspath('')
in_filename = os.path.join(base_dir, 'img\\barvaz.jpg')
out_filename  = os.path.join(base_dir, 'img\\barvaz1.jpg')
img = imread(in_filename)


In [6]:
out = Cropper.crop_c(img, 0.9)
imwrite(out_filename, out)

  0%|                                                                                           | 0/44 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 2)

In [5]:
from PIL import Image
in_filename = os.path.join(base_dir, 'img\\barvaz.jpg')
img = Image.open(in_filename).convert('LA')
img.save('barvaz.png')

In [12]:
import cv2
import numpy as np

drawing = False # true if mouse is pressed
mode = True # if True, draw rectangle. Press 'm' to toggle to curve
ix,iy = -1,-1

# mouse callback function
def draw_circle(event,x,y,flags,param):
    global ix,iy,drawing,mode
    color = (255,255,255)
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix,iy = x,y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
            if mode == True:
                cv2.rectangle(img,(ix,iy),(x,y),color,-1)
            else:
                cv2.circle(img,(x,y),3,color,-1)
                cv2.circle(blackAndWhite,(x,y),3,color,-1)


    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        if mode == True:
            cv2.rectangle(img,(ix,iy),(x,y),color,-1)
        else:
            cv2.circle(img,(x,y),3,color,-1)
            cv2.circle(blackAndWhite,(x,y),3,color,-1)

            
            
in_filename = os.path.join(base_dir, 'img\\barvaz.jpg')
img = imread(in_filename)
r, c = img.shape[: 2]
blackAndWhite  = np.zeros((r, c, 3))

cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_circle)

while(1):
    cv2.imshow('image',img)
    k = cv2.waitKey(1) & 0xFF
    if k == ord('m'):
        mode = not mode
    elif k == 27:
        blackAndWhite = cv2.cvtColor(blackAndWhite, cv2.COLOR_BGR2GRAY)
        cv2.imwrite("mask.jpg", blackAndWhite)
        break

cv2.destroyAllWindows()

error: OpenCV(4.0.0) c:\projects\opencv-python\opencv\modules\imgproc\src\color.hpp:261: error: (-2:Unspecified error) in function '__cdecl cv::CvtHelper<struct cv::Set<3,4,-1>,struct cv::Set<1,-1,-1>,struct cv::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Unsupported depth of input image:
>     'VDepth::contains(depth)'
> where
>     'depth' is 6 (CV_64F)


In [ ]:
out = Cropper.remove_object(img,x_y_objectRemoval)
imwrite(out_filename, out)

the minumum path is :-839638.0
the minumum path is :-828762.0
the minumum path is :-788422.0
the minumum path is :-679532.0
the minumum path is :-645762.0
the minumum path is :-585478.0
the minumum path is :-497744.0
the minumum path is :-478304.0
the minumum path is :-416472.0
the minumum path is :-352028.0
the minumum path is :-327422.0
the minumum path is :-301232.0
the minumum path is :-281238.0
the minumum path is :-230360.0
the minumum path is :-180732.0
the minumum path is :-174140.0
the minumum path is :-150730.0
the minumum path is :-101584.0
the minumum path is :-96232.0
the minumum path is :-60192.0
